In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/Triple_Construction

/content/drive/MyDrive/Colab Notebooks/Triple_Construction


In [ ]:
ls

 1_triple_construction.ipynb
 2_triple_count_256seg.ipynb
 2_triple_count.ipynb
'Copy of topic4_30000sents_entityData_finalModel.csv'
 entity_data.csv
 entity_dataset.csv
 kg_df_finalResult_256seg.csv
 kg_df_finalResult_with_6OrMoreDoc.csv
 original_triple_construction_test2.ipynb
 preprocessed_topic4_triples_256seg.csv
 preprocessed_topic4_triples.csv
 topic4_5000sents_triples.csv
 triple_group_by_doc_count.csv
 triple_with_doc_count.csv
 verb_data.csv
 verb_dataset.csv


In [ ]:
import time

start_time = time.time()

In [ ]:
import pandas as pd
import numpy as np

### Load dataset and tokenize

In [ ]:
triple_df = pd.read_csv('preprocessed_topic4_triples.csv')
triple_df.head()

,Unnamed: 0,doc_id,sent_id,triple,head_idx,verb_idx,tail_idx,head_str,head_lemma,head_type,tail_str,tail_lemma,tail_type,main_verb,lemma_verb,cluster_verb
0,0,0.0,0.0,submacular hemorrhage relate sars-cov-2,10.0,11.0,14.0,submacular hemorrhage,submacular hemorrhage,DISEASE_OR_SYNDROME,sars-cov-2,sars-cov-2,CORONAVIRUS,related,relate,relate
1,1,0.0,0.0,submacular hemorrhage relate vaccine,10.0,11.0,15.0,submacular hemorrhage,submacular hemorrhage,DISEASE_OR_SYNDROME,vaccines,vaccine,CHEMICAL,related,relate,relate
2,2,0.0,0.0,age-related relate sars-cov-2,5.0,11.0,14.0,age-related,age-related,GENE_OR_GENOME,sars-cov-2,sars-cov-2,CORONAVIRUS,related,relate,relate
3,3,0.0,0.0,age-related relate vaccine,5.0,11.0,15.0,age-related,age-related,GENE_OR_GENOME,vaccines,vaccine,CHEMICAL,related,relate,relate
4,4,0.0,0.0,macular relate sars-cov-2,6.0,11.0,14.0,macular,macular,TISSUE,sars-cov-2,sars-cov-2,CORONAVIRUS,related,relate,relate


In [ ]:
total_doc_no = triple_df['doc_id'].tolist()[-1]
print('total doc no: ', total_doc_no)

total doc no:  495.0


In [ ]:
print('len: ', len(triple_df))

len:  23825


In [ ]:
# drop the triples that doesn't have cluster_verb.
# because I skipped lemma that doesn't have synset meaning that they are not general & common verb. 

In [ ]:
triple_df.drop(triple_df[triple_df.isna().any(axis=1)].index, axis=0, inplace=True)

In [ ]:
print('len: ', len(triple_df))

len:  23825


Decide which one to use ...

In [ ]:
triple_type = 'triple'

In [ ]:
triple_occur_df = triple_df.groupby(triple_type)['doc_id'].apply(list).reset_index(name='doc_list')

In [ ]:
triple_occur_df['unique_doc'] = triple_occur_df['doc_list'].apply(lambda x: list(set(x)))

In [ ]:
triple_occur_df['doc_count'] = triple_occur_df['unique_doc'].apply(lambda x: len(x))

In [ ]:
triple_occur_df.head()

,triple,doc_list,unique_doc,doc_count
0,- use covid-19,[429.0],[429.0],1
1,-- induce infection,[447.0],[447.0],1
2,-related relate sars-cov-2,[0.0],[0.0],1
3,-related relate vaccine,[0.0],[0.0],1
4,-specific detect th1,[28.0],[28.0],1


In [ ]:
# maximum duplicated document numbers 

print('maximum duplicated document numbers: ', max(triple_occur_df['doc_count'].tolist()))

maximum duplicated document numbers:  8


In [ ]:
# group by doc_count

doc_group = triple_occur_df.groupby('doc_count')

In [ ]:
doc_df = doc_group['triple'].apply(list).reset_index(name='triple_list')

In [ ]:
doc_df

,doc_count,triple_list
0,1,"[- use covid-19, -- induce infection, -related..."
1,2,"[100 receive participant, 183 develop hba1c, 2..."
2,3,"[antibody bind vaccine, antibody induce sars-c..."
3,4,"[covid-19 induce patient, covid-19 receive vac..."
4,5,"[covid-19 induce vaccine, patient induce vacci..."
5,6,"[covid-19 need vaccine, covid-19 receive patie..."
6,7,"[patient receive covid-19, sars-cov-2 induce v..."
7,8,[patient receive vaccine]


In [ ]:
## determine document threshold
doc_count = 6

In [ ]:
kg_df = triple_occur_df[triple_occur_df['doc_count'] >= doc_count][[triple_type, 'unique_doc', 'doc_count']]

In [ ]:
kg_df

,triple,unique_doc,doc_count
6260,covid-19 need vaccine,"[128.0, 233.0, 44.0, 338.0, 244.0, 55.0]",6
6397,covid-19 receive patient,"[258.0, 423.0, 80.0, 212.0, 349.0, 31.0]",6
6401,covid-19 receive sars-cov-2,"[423.0, 169.0, 493.0, 151.0, 282.0, 349.0]",6
13920,patient receive covid-19,"[130.0, 67.0, 134.0, 391.0, 137.0, 147.0, 84.0]",7
13971,patient receive vaccine,"[67.0, 134.0, 391.0, 137.0, 41.0, 42.0, 406.0,...",8
15970,sars-cov-2 induce vaccine,"[385.0, 264.0, 330.0, 48.0, 435.0, 277.0, 317.0]",7


In [ ]:
triple_occur_df[triple_occur_df['doc_count'] == 3][[triple_type, 'doc_count']]

,triple,doc_count
2709,antibody bind vaccine,3
2799,antibody induce sars-cov-2,3
2829,antibody neutralize vaccine,3
5385,coronavirus associate covid-19,3
5434,coronavirus compare covid-19,3
5536,coronavirus induce covid-19,3
5546,coronavirus induce sars-cov-2,3
5549,coronavirus induce vaccine,3
5934,covid-19 compare patient,3
6155,covid-19 increase vaccine,3


In [ ]:
# save the triples for visualization, analyzing

kg_df.to_csv('kg_df_finalResult_with_6OrMoreDoc.csv')

In [ ]:
triple_occur_df.to_csv('triple_with_doc_count.csv')

In [ ]:
doc_df.to_csv('triple_group_by_doc_count.csv')

In [ ]:
end_time = time.time()

print('Estimate runtime: ', (end_time-start_time)/60, ' minutes.')

Estimate runtime:  0.06765611171722412  minutes.


In [ ]:
### for 5000 sentences

# Estimate runtime:  0.19963531891504924  minutes.